In [1]:
config = {
    "user": "root",
    "password": "tseinfo",
    "database": "pri_database",
    "host": "localhost",
    "port": 3306
}

In [2]:
posts_query = "SELECT post_id, title, date, url, book_title FROM posts"
keywords_of_a_post_query = ("SELECT KEYWORD_NAME FROM posts_keywords pk "
                            "JOIN keywords k "
                            "ON pk.KEYWORD_ID = k.KEYWORD_ID "
                            "WHERE pk.POST_ID = {};")
delete_post_query = "DELETE FROM posts WHERE POST_ID = {}"

categories_query = "SELECT category_id, category_name, subcategory_id FROM categories"
add_categories_query = 'INSERT INTO categories (CATEGORY_NAME) VALUES ("{}")'

set_post_category_query = "INSERT INTO posts_categories (POST_ID, CATEGORY_ID) VALUES ({}, {})"
delete_post_category_query = "DELETE FROM posts_categories WHERE POST_ID = {}"

In [3]:
# You need to pip install mysql
try:
    import mysql.connector
except ModuleNotFoundError:
    import sys
    print("Error: You need to `pip install mysql-connector-python`", file=sys.stderr)

In [4]:
cursor = None
cnx = None

def connect():
    global cursor, cnx
    #print("Connecting to database...")
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    cnx.autocommit = True

def disconnect():
    global cursor, cnx
    cursor.close()
    cnx.close()

def query(query, fetch=True, close_connexion=True, limit_count=None, limit_offset=0):
    global cursor, cnx
    if limit_count != None:
        query = query + " LIMIT {}, {}".format(limit_offset, limit_count)
    try:
        cursor.execute(query)
    except AttributeError: # cursor is None
        connect()
        cursor.execute(query)
    except mysql.connector.errors.ProgrammingError: # cursor is not connected
        connect()
        cursor.execute(query)
    finally:
        #print("executed query:\n", query) # useful for debugging
        if fetch:
            result = cursor.fetchall()
        if close_connexion:
            disconnect()
        if fetch:
            return result

In [5]:
def get_prints(limit=None):
    Ids, Titles, Book_titles, Keywords = [], [], [], []
    results = query(posts_query, limit_count=limit, close_connexion=False)
    for (post_id, title, date, url, book_title) in results:
        #print("{:%d %b %Y}: {}".format(date, title))
        #print(" | title:", title)
        #print(" | book title:", book_title)
        keywords = []
        kw_results = query(keywords_of_a_post_query.format(post_id), close_connexion=False)
        for fields in kw_results:
            keywords.append(fields[0]) # fields[0] = keyword field
        #print(" |", len(keywords), "keywords:", " ; ".join(keywords))
        Ids.append(post_id)
        Titles.append(title if title else "")                # if None, put "" instead
        Book_titles.append(book_title if book_title else "") # if None, put "" instead
        Keywords.append(keywords if keywords else "")        # if None, put "" instead
    disconnect()
    return Ids, Titles, Book_titles, Keywords

In [6]:
def get_categories(debug = False):
    Category_ids = []
    Category_names = []
    
    categories = query(categories_query)
    for (id_, name, subcategories) in categories:
        if debug:
            print("{}: {}".format(id_, name))
        Category_ids.append(id_)
        Category_names.append(name)
    return {name: id_ for name, id_ in zip(Category_names, Category_ids)}

In [7]:
def add_category(name):
    query(add_categories_query.format(name), fetch=False)

def add_categories(Names):
    for name in Names:
        add_category(name)

In [8]:
Categories = ["Others", "Crypto"]
Database_categories = get_categories()
for category in Categories:
    if category not in Database_categories:
        print("Warning: category '{}' not found in the database. Adding it to the 'categories' table.".format(category))
        add_category(category)
Database_categories = get_categories() # get_categories(debug=True) to see the categories in the database

In [9]:
def set_print_category(post_id, category_name, delete_post_if_category_is="Other", close_connexion=True):
    # delete current value
    query(delete_post_category_query.format(post_id), fetch=False, close_connexion=False)
    # create new value, or delete the post if delete_post_if_category_is == category_name
    if delete_post_if_category_is != None and category_name == delete_post_if_category_is:
            # if delete_post_if_category_is is specified and match the post category, delete the post
        query(delete_post_query, fetch=False, close_connexion=close_connexion)
    else:
        assert category_name in Categories, "category {} is not in the list Categories {}, but it should".format(category_name, Categories)
        category_id = Database_categories[category_name]
        query(set_post_category_query.format(post_id, category_id), fetch=False, close_connexion=close_connexion)

def set_print_categories(Post_ids, Category_names):
    assert len(Post_ids) == len(Category_names), "There should be one category per id, but there is not"
    for i in range(len(Post_ids)):
        set_print_category(Post_ids[i], Category_names[i], close_connexion=False)
    disconnect()

In [10]:
set_print_category(68352, "Crypto")

In [11]:
def mock_classifier(x_titles, x_keywords, x_booktitles):
    import random
    assert len(x_titles) == len(x_booktitles) and len(x_titles) == len(x_keywords)
    return [random.choice(["Others", "Crypto"]) for i in range(len(x_titles))]

In [12]:
import time

t1 = time.time()
x_ids, x_titles, x_booktitles, x_keywords = get_prints()
prediction = mock_classifier(x_titles, x_keywords, x_booktitles)
print("took", round(time.time() - t1), "s to load", len(prediction), "prints")

t1 = time.time()
set_print_categories(x_ids, prediction)
print("took", round(time.time() - t1), "s to save", len(prediction), "predictions")

took 2 s to load 10500 prints
took 84 s to save 10500 predictions


In [13]:
import time
from Predict import predict

t1 = time.time()
x_ids, x_titles, x_booktitles, x_keywords = get_prints()
print("took", round(time.time() - t1), "s to load", len(x_ids), "prints")

t1 = time.time()
prediction = predict(x_titles, x_keywords, x_booktitles)
print("took", round(time.time() - t1), "s to predict", len(prediction), "categories")

t1 = time.time()
#set_print_categories(x_ids, prediction)
print("took", round(time.time() - t1), "s to save", len(prediction), "predictions")

C:\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


took 2 s to load 10500 prints
took 93 s to predict 10500 categories
took 0 s to save 10500 predictions
